For this project we aim to use the EC2 Instance Connect End Point to connect to resources in the VPC (Virtual Private Cloud) using the AWS CLI. Once set up an IAM User will be created with programmatic access which will be configured to our computer. This will allow us to use the EC2 Instance connect endpoint to connect to resources in the private subnet and the public subnet of our VPC using AWS CLI

In [ ]:
Requirements: 
- homebrew 
- iterm2
- AWS CLI

AWS - IAMS 
(Creating new Administrative Access User)

We're going to create an administrative Access User Using the following steps: 
1. Create User

<img src="Images/UsersSelect.png" alt="Drawing" style="width: 300px;"/>

![CreateUser](Images/CreateUser.png)

2. Once created Attach required policies: 
![UserName](Images/UserName.png)
![AttachPolicies](Images/AttachPolicies.png)

Creating Secret Access Key for the IAM User 
![ClickUser](Images/ClickUser.png)
![CreateAccessKey](Images/CreateAccessKey.png)
![CLIAttribute](Images/CLIAttribute.png)
![AccessKeyShow](Images/AccessKeyShow.png)

Configuring user's access Key ID and secret access key on the computer to use the AWS CLI and EC2 Instance to connect to any resources in the VPC


On Terminal: 

aws configure

It will request for the access Key 

![aws configure](Images/awsConfigure.png)

Credentials can be found in AWS folder 
on terminal 

> cd 
> ls -a (for hidden folder to be revealed as well)

after inputting credentials AWS creates a directory .aws and store credentials in there


>cd .aws

will show us there's 2 files the credentials file and the config file 
to view inside the config file 

>cat config
[default]
region = eu-west-2

We can see the configuration has created a default where the region has been stored